# CUNEF MUCD 2021/2022  
## Machine Learning
## Análisis de Siniestralidad de Automóviles

### Autores:
- Andrés Mahía Morado
- Antonio Tello Gómez


# Modelo Base y Naive Bayes

En este apartado construimos el modelo base que nos servirá como referencia para comprobar el desempeño del resto de modelos. Por otro lado, también vamos entrenar un modelo Naive Bayes otro modelo sencillo que también nos servirá como referencia. 

In [1]:
#Librerías
import pandas as pd
import numpy as np

from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, \
                            silhouette_score, recall_score, precision_score, make_scorer, \
                            roc_auc_score, f1_score, precision_recall_curve
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, roc_auc_score, \
                            classification_report, confusion_matrix
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix

from sklearn.dummy import DummyClassifier 
from sklearn.naive_bayes import GaussianNB

import pickle
import warnings
warnings.filterwarnings('ignore')

In [2]:
from aux_func import evaluate_model

In [3]:
xtrain = pd.read_parquet("../data/xtrain.parquet")
ytrain = pd.read_parquet("../data/ytrain.parquet")
xtest = pd.read_parquet("../data/xtest.parquet")
ytest = pd.read_parquet("../data/ytest.parquet")

# Modelo Base

Un modelo base emplea un valor fijo o valores aleatorios (Normalmente en función de algún estadistico de la distribución de la variable objetivo) para realizar la predicción de los valores del test. 
En nuestro caso hemos empleado la opción 'stratified' que genera predicciones aleatorias respetando la distribución de la variable objetivo.  


In [4]:
%%time
modelo_base = DummyClassifier(strategy='stratified', random_state=0)
modelo_base.fit(xtrain, ytrain)

Wall time: 395 ms


DummyClassifier(random_state=0, strategy='stratified')

In [5]:
with open('../models/modelo_base.pickle', 'wb') as f:
    pickle.dump(modelo_base, f)

In [6]:
# Para no tener que ejecutar, saltarse el fit y ejecutar a partir de aquí
with open('../models/modelo_base.pickle', 'rb') as f:
    clf = pickle.load(f)

In [7]:
ypred = modelo_base.predict(xtest)
ypred_proba = modelo_base.predict_proba(xtest)
evaluate_model(ytest,ypred,ypred_proba)

ROC-AUC score of the model: 0.49958685184478924
Accuracy of the model: 0.9702494339127026

Classification report: 
              precision    recall  f1-score   support

           0       0.98      0.99      0.98    997197
           1       0.01      0.01      0.01     15456

    accuracy                           0.97   1012653
   macro avg       0.50      0.50      0.50   1012653
weighted avg       0.97      0.97      0.97   1012653


Confusion matrix: 
[[982308  14889]
 [ 15238    218]]



Algo muy significativo de nuestro modelo base es que obtenemos un accuracy, la métrica por excelencia para evaluar los modelos de ML, del 97.02%. Si predijéramos que todo nuestro test pertenece a la clase 0 (strategy='most_frequent') obtendríamos un accuracy aún mayor, del 98.47%. Sin embargo, no estaríamos acercándonos de ninguna manera a nuestro objetivo que es predecir el máximo posible de la clase 1 sin renunciar a acertar en la clase 0. Por tanto, el accuracy no será una métrica muy útil a la hora de evaluar nuestros modelos.

# Naive Bayes

El Naive Bayes es un clasificador basado en la aplicación del teorema de Bayes con la hipótesis de independencia condicional entre variables predictoras. Hipótesis conocida como ingenua, _naive_.  
En nuestro caso hemos estimado un Gaussian Naive Bayes en el que asumimos que probabilidad de las variables predictoras sigue una distribución normal.  
Este es el modelo más sencillo que tiene en cuenta las variables predictoras por ello también será una referencia muy útil para comparar el resto de modelos.

In [9]:
%%time
naive_bayes = GaussianNB()
naive_bayes = naive_bayes.fit(xtrain, ytrain)

Wall time: 5.49 s


In [10]:
with open('../models/GNB.pickle', 'wb') as f:
    pickle.dump(naive_bayes, f)

In [11]:
# Para no tener que ejecutar, saltarse el fit y ejecutar a partir de aquí
with open('../models/GNB.pickle', 'rb') as f:
    clf = pickle.load(f)

In [12]:
ypred = naive_bayes.predict(xtest)
ypred_proba = naive_bayes.predict_proba(xtest)

In [13]:
evaluate_model(ytest,ypred,ypred_proba)

ROC-AUC score of the model: 0.7873892350291511
Accuracy of the model: 0.8959248627121037

Classification report: 
              precision    recall  f1-score   support

           0       0.99      0.90      0.94    997197
           1       0.07      0.48      0.12     15456

    accuracy                           0.90   1012653
   macro avg       0.53      0.69      0.53   1012653
weighted avg       0.98      0.90      0.93   1012653


Confusion matrix: 
[[899825  97372]
 [  8020   7436]]



## Ajuste del umbral de predicción

In [14]:
# keep probabilities for the positive outcome only
yhat = ypred_proba[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, yhat)

gmeans = np.sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))

ypred_new_threshold = (ypred_proba[:,1]>thresholds[ix]).astype(int)
evaluate_model(ytest,ypred_new_threshold,ypred_proba)

Best Threshold=0.003422, G-Mean=0.718
ROC-AUC score of the model: 0.7873892350291511
Accuracy of the model: 0.732287367933537

Classification report: 
              precision    recall  f1-score   support

           0       0.99      0.73      0.84    997197
           1       0.04      0.70      0.07     15456

    accuracy                           0.73   1012653
   macro avg       0.52      0.72      0.46   1012653
weighted avg       0.98      0.73      0.83   1012653


Confusion matrix: 
[[730680 266517]
 [  4583  10873]]

